In [ ]:
#!/usr/bin/env python3
#  -*- coding: utf-8 -*-
"""A simple bot script, built on Flask, that demonstrates posting a
card, and handling the events generated when a user hits the Submit button.
A bot must be created and pointed to this server in the My Apps section of
https://developer.webex.com.  The bot's Access Token should be added as a
"WEBEX_TEAMS_ACCESS_TOKEN" environment variable on the web server hosting this
script.
This script must expose a public IP address in order to receive notifications
about Webex events.  ngrok (https://ngrok.com/) can be used to tunnel traffic
back to your server if your machine sits behind a firewall.
The following environment variables are needed for this to run
* WEBEX_TEAMS_ACCESS_TOKEN -- Access token for a Webex bot
* WEBHOOK_URL -- URL for Webex Webhooks (ie: https://2fXX9c.ngrok.io)
* PORT - Port for Webhook URL (ie: the port param passed to ngrok)
This sample script leverages the Flask web service micro-framework
(see http://flask.pocoo.org/).  By default the web server will be reachable at
port 5000 you can change this default if desired (see `flask_app.run(...)`).
In our app we read the port from the PORT environment variable.
Upon startup this app create webhooks so that our bot is notified when users
send it messages or interact with any cards that have been posted.   In
response to any messages it will post a simple form filling card.  In response
to a user submitting a form, the details of that response will be posted in
the space.
This script should support Python versions 3.6+ only.
Copyright (c) 2016-2020 Cisco and/or its affiliates.
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import json
import os
import sys
from urllib.parse import urljoin
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
import statistics
import matplotlib.pyplot as plt
from datetime import date

from flask import Flask, request

from webexteamssdk import WebexTeamsAPI, Webhook


# Script metadata
__author__ = "Healkan Cheung"
__author_email__ = "heacheunr@cisco.com"
__code_reuse_from__ = ["JP Shipherd jshipher@cisco.com","Chris Lunsford <chrlunsf@cisco.com>"]
__copyright__ = "Copyright (c) 2016-2020 Cisco and/or its affiliates."
__license__ = "MIT"


# Constants
WEBHOOK_NAME = "botWithCardExampleWebhook"
WEBHOOK_URL_SUFFIX = "/events"
MESSAGE_WEBHOOK_RESOURCE = "messages"
MESSAGE_WEBHOOK_EVENT = "created"
CARDS_WEBHOOK_RESOURCE = "attachmentActions"
CARDS_WEBHOOK_EVENT = "created"
output_file = 'LDOS-out.txt'
input_file = 'LDOS-temp.txt'



# Module variables
webhook_url = os.environ.get("WEBHOOK_URL", "")
port = int(os.environ.get("PORT", 0))
access_token = os.environ.get("WEBEX_TEAMS_ACCESS_TOKEN", "")
if not all((webhook_url, port, access_token)):
    print(
        """Missing required environment variable.  You must set:
        * WEBHOOK_URL -- URL for Webex Webhooks (ie: https://2fXX9c.ngrok.io)
        * PORT - Port for Webhook URL (ie: the port param passed to ngrok)
        * WEBEX_TEAMS_ACCESS_TOKEN -- Access token for a Webex bot
        """
    )
    sys.exit()

# Initialize the environment
# Create the web application instance
flask_app = Flask(__name__)
# Create the Webex Teams API connection object
api = WebexTeamsAPI()
# Get the details for the account who's access token we are using
me = api.people.me()


# Helper functions
def delete_webhooks_with_name():
    """List all webhooks and delete webhooks created by this script."""
    for webhook in api.webhooks.list():
        if webhook.name == WEBHOOK_NAME:
            print("Deleting Webhook:", webhook.name, webhook.targetUrl)
            api.webhooks.delete(webhook.id)


def create_webhooks(webhook_url):
    """Create the Webex Teams webhooks we need for our bot."""
    print("Creating Message Created Webhook...")
    webhook = api.webhooks.create(
        resource=MESSAGE_WEBHOOK_RESOURCE,
        event=MESSAGE_WEBHOOK_EVENT,
        name=WEBHOOK_NAME,
        targetUrl=urljoin(webhook_url, WEBHOOK_URL_SUFFIX)
    )
    print(webhook)
    print("Webhook successfully created.")

    print("Creating Attachment Actions Webhook...")
    webhook = api.webhooks.create(
        resource=CARDS_WEBHOOK_RESOURCE,
        event=CARDS_WEBHOOK_EVENT,
        name=WEBHOOK_NAME,
        targetUrl=urljoin(webhook_url, WEBHOOK_URL_SUFFIX)
    )
    print(webhook)
    print("Webhook successfully created.")





def respond_to_message(webhook):
    """Respond to a message to our bot"""

    # Some server side debugging
    room = api.rooms.get(webhook.data.roomId)
    message = api.messages.get(webhook.data.id)
    person = api.people.get(message.personId)
    if message.text:
        print(
        f"""
        NEW MESSAGE IN ROOM '{room.title}'
        FROM '{person.displayName}'
        MESSAGE '{message.text}'
        """
        )
    elif message.files:
        print(
            f"""
            NEW MESSAGE IN ROOM '{room.title}'
            FROM '{person.displayName}'
            MESSAGE '{message.files}'
            """
        )


    # This is a VERY IMPORTANT loop prevention control step.
    # If you respond to all messages...  You will respond to the messages
    # that the bot posts and thereby create a loop condition.
    if message.personId == me.id:
        # Message was sent by me (bot); do not respond.
        return "OK"

    # Message was sent by someone else; parse message and respond.
    # Check if message is a file.  If it is a file, then get the url to retrieve file
    elif message.files:         
        api.messages.create(
            room.id,
            text="Preparing LDOS info...",
        )
        url = message.files[0]

        # Send a GET request to retrieve contents of file and save it to a temp file
        payload = {}
        headers = {
            'Accept': 'application/json',
            'Authorization': 'Bearer ' + access_token
        }
        response = requests.request("GET", url, headers=headers, data = payload)
        outf = open(input_file,'w')
        for i in response.text:
            outf.writelines(i)
        outf.close()

        # Call up function to use Cisco APIs to obtain LDOS info and save to output file
        # Save the LDOS content to a dictionary
        inventory = lookup_ldos()
        
        print(inventory)
        
        replacement = show_ldos(inventory, 2020,2025)

        # Generate graphs to show LDOS quantity, median age and locations from 2020 to 2025
        plot_ldos_by_quantity(inventory,2020,2025)
        plot_ldos_by_age(inventory,2020,2025)
        plot_ldos_by_site(inventory,2020,2025)

        api.messages.create(
             room.id,
            text="Finish",
        )
        api.messages.create(
            room.id,
            text="Getting Outputs",
        ) 
        
        # Sending LDOS text outputs and plots to Teams room
        api.messages.create(room.id,files=['LDOS-out.txt'])
        api.messages.create(room.id,files=['LDOS-by-year.png'])     
        api.messages.create(room.id,files=['LDOS-by-age.png'])   
        api.messages.create(room.id,files=['LDOS-by-site.png'])  

        estimate_id = create_bom(replacement,2020,2025)
        api.messages.create(
            room.id,
            text="New CCW Estimate " + estimate_id + " at https://tools.cisco.com/POE/Commerce/estimate#!&report=all "
        )

    # Message is a text.  Display instruction to upload file
    else:
        api.messages.create(
            room.id,
            text="Please attach a text file with serial numbers (one per line; up to 10,000 lines)",
        )
    return "OK"

    

    

# Core bot functionality
# Webex will post to this server when a message is created for the bot
# or when a user clicks on an Action.Submit button in a card posted by this bot
# Your Webex Teams webhook should point to http://<serverip>:<port>/events
@flask_app.route("/events", methods=["POST"])
def webex_teams_webhook_events():
    """Respond to inbound webhook JSON HTTP POST from Webex Teams."""
    # Create a Webhook object from the JSON data
    webhook_obj = Webhook(request.json)

    # Handle a new message event
    if (webhook_obj.resource == MESSAGE_WEBHOOK_RESOURCE
            and webhook_obj.event == MESSAGE_WEBHOOK_EVENT):
        respond_to_message(webhook_obj)


    # Ignore anything else (which should never happen
    else:
        print(f"IGNORING UNEXPECTED WEBHOOK:\n{webhook_obj}")

    return "OK"


# Get Cisco API Auth token
def getnewtoken():
    
    # Get API authn client id and secret from file api_cred
    from api_cred import client_id
    from api_cred import client_secret

    # Remember the current time.  This is used to determine if token needs to be refreshed.
    now = time.time()

    # URL for REST call to generate new token
    url = "https://cloudsso.cisco.com/as/token.oauth2?client_id=" + client_id + "&grant_type=client_credentials&client_secret=" + client_secret

    payload = {}
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': 'PF=79XZT8hLaZ8I29ms2LsKo7'
    }

    response = requests.request("POST", url, headers=headers, data = payload)
  
    tresponse = json.loads(response.text)
    token = tresponse["access_token"]

    # Saving token to be used by Cisco APIs
    token = 'Bearer ' + token
    
    return token,now

# Check if API token needs to be refreshed
def refreshtoken(token,now):

    # Get current time and check if the time since the auth token was created. 
    later = time.time()
    difference = int(later - now)

    #  If time since token was created is greater than 3500 seconds, then get a new token. 
    #  3599 seconds is the documented expiration period, but we use 3500 sec to be safe.
    if difference >= 3500:
        print('Refresh new token')
        token,now = getnewtoken()
    return token,now

# Function to call CCW-R API
def ccw_r_api(serial,token):
    url = "https://api.cisco.com/ccw/renewals/api/v1.0/search/lines"

    # Payload contains a list of up to 20 serial numbers.  
    payload = '{\n    \"serialNumbers\": [\n        ' + serial + '\n    ],\n    \"offset\": 0,\n    \"limit\": 1000,\n    \"configurations\": true\n}'
    headers = {
      'Content-Type': 'application/json',
      'Authorization': token,
      'Accept-Language': 'en_US',
      'Request-id': '78909092'
    }

    response = requests.request("POST", url, headers=headers, data = payload)

    # saving CCW-R API output in a JSON dictionary
    ccw_r_response = json.loads(response.text)

    return ccw_r_response
    


# Function to call EOX API
def eox_api(pid,token):

    # Retrieving EOX info for a serial number
    url = "https://api.cisco.com/supporttools/eox/rest/5/EOXByProductID/1/" + pid + "?responseencoding=json"

    payload = {}
    headers = {
      'Accept': 'application/json',
      'Authorization': token
    }

    # Code to retry API call if timing out
    with requests.Session() as s:
        retries = Retry(
            total=10,
            backoff_factor=0.2,
            status_forcelist=[500, 502, 503, 504])

    s.mount('http://', HTTPAdapter(max_retries=retries))
    s.mount('https://', HTTPAdapter(max_retries=retries))
    response = s.get(url,headers=headers, data = payload)
    
    # saving EOX API output in a JSON dictionary      
    eox_api_response = json.loads(response.text)
        
    return eox_api_response


def show_ldos(inventory,first_year,last_year):
    outfile = open(output_file,'w')
    replacement={}
    
    for year in range(first_year, last_year+1):
        replacement[str(year)]={}
        years_shipped = []
        outfile.writelines(["\n LDOS for Year ", str(year),"\n"])
        num_LDOS = 0
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                num_LDOS += 1
                outfile.writelines(["Serial: ",dev["Serial"]," Site: ", dev["Site"], " Product: ", dev["Product"]," LDOS Date: ",dev["LDOSDate"]," Replacement: ",dev["Replacement"], " Ship Date: ", dev["ShipDate"],"\n"])

                if dev["Replacement"] in replacement[str(year)].keys():
                    replacement[str(year)][dev["Replacement"]] += 1
                else:
                    replacement[str(year)][dev["Replacement"]] = 1
                if dev["ShipDate"] != "":
                    yr = int(dev["ShipDate"][0:4])
                    mo= int(dev["ShipDate"][5:7])
                    day= int(dev["ShipDate"][8:10])
                    f_date = date(yr, mo, day)
                    l_date = date(2020, 10, 19)
                    delta = l_date - f_date
                    years_shipped.append(delta.days/365)
        outfile.writelines(["\nReplacement ", str(year),": ", str(replacement[str(year)]),"\n"])
        if years_shipped:
            outfile.writelines(["\n Median age of LDOS gear = ", str(round(statistics.median(years_shipped),2)), " years.  ", "Oldest gear is: ", str(round(max(years_shipped),2)), " years old.","\n\n" ] )

    # Showing devices with no LDOS
    years_shipped = []
    outfile.writelines(["\n No LDOS \n"])
    for dev in inventory:
        if dev["LDOSDate"] == "":

            outfile.writelines(["Serial: ",dev["Serial"]," Site: ", dev["Site"], " Product: ", dev["Product"]," LDOS Date: ",dev["LDOSDate"]," Replacement: ",dev["Replacement"], " Ship Date: ", dev["ShipDate"],"\n"])

            if dev["ShipDate"] != "":
                yr = int(dev["ShipDate"][0:4])
                mo= int(dev["ShipDate"][5:7])
                day= int(dev["ShipDate"][8:10])
                f_date = date(yr, mo, day)
                l_date = date(2020, 10, 19)
                delta = l_date - f_date
                years_shipped.append(delta.days/365)
    if years_shipped:
        outfile.writelines(["\n Median age of LDOS gear = ", str(round(statistics.median(years_shipped),2)), " years.  ", "Oldest gear is: ", str(round(max(years_shipped),2)), " years old.","\n\n" ] )
            

    outfile.close()
    return replacement


# Function to read lines from input file, get LDOS info from Cisco REST APIs and save LDOS info into an output file 
def get_ldos(num,infile,output,token,now,LDOS_buffer):

    # 'serial_block' is dictionary to store serial number key to site location value
    serial_block={}

    
    # Read 'num' number of lines from input file.  Save each serial number & site id as a key value pair. 
    # This is used to check if CCW-R output entry matches the serial number
    for r in range(num):
        serial = infile.readline()
        if len(serial.split()) == 2:
            serial_block[serial.split()[0]] = serial.split()[1]
        else:
            serial_block[serial.split()[0]] = "Unknown"
        
    # Creating a string of just the serial numbers.  This is used in the CCW-R API payload
    s_block = str([i for i in serial_block]).strip('[]')

    # Before calling any Cisco API, check if auth token needs to be refreshed.
    token,now = refreshtoken(token,now)

    # Calling CCW-R API for the block of serial numbers and save the output in a dictionary  
    ccw_r_response = ccw_r_api(s_block,token)



    # Iterate through the CCW-R API output
    for dev in ccw_r_response['instances']:

        # Check if the Serial Number field is in each CCW-R output entry.     
        if ("serialNumber" in dev):
            
            # LDOS data for each device will be saved in this dictionary
            device = {}
            
            # Check if the serial number in the CCW-R entry matches any of the serial numbers from the input file
            # If then save the relevant fields to the 'device' dictonary which will be save to output file later.
            if dev["serialNumber"] in serial_block:
                # Get location field from CCW-R output unless it is included in the inut file
                if serial_block[dev["serialNumber"]] == "Unknown":
                    device["Site"] = dev["endCustomer"]["address"]["city"]+','+dev["endCustomer"]["address"]["country"]
                else:
                    device["Site"] = serial_block[dev["serialNumber"]]
                
                # Removing the serial number so if there is duplicate entry in another CCW-R output entry, then it will not match and we don't get a duplicate
                del serial_block[dev["serialNumber"]]
                
                device["Serial"] = dev["serialNumber"]
                device["Product"] = dev["product"]["number"]
                
                # checking if device serial has a LDOS entry.  If so call the Cisco EOX API to get replacement product ID
                if "lastDateOfSupport" in dev:
                    
                    # before API call, let's check if auth token needs to be refreshed
                    token,now = refreshtoken(token,now)
                    
                    # checking if there is already a LDOS product entry in the temporary dictionary.  
                    # If there is then get the LDOS date and replacement from this dictionary so we don't need to call EOX API unnecessarily
                    if device["Product"] in LDOS_buffer:
                        device["LDOSDate"] = LDOS_buffer[device["Product"]]["LDOSDate"]
                        device["Replacement"] =  LDOS_buffer[device["Product"]]["Replacement"]

                    # If the product id is not in the temporary dictionary, then call the EOX API and save the info
                    else:
                        eol_prod = dev["product"]["number"]
                        replacement_is_ldos = True
                        ldos_date =  ""
                        
                        # Need to check if the replacement product is itself LDOS
                        while replacement_is_ldos:
                            eox_api_response = eox_api(eol_prod,token)
                    
                            if (eox_api_response["EOXRecord"][0]["LastDateOfSupport"]["value"] != "") and \
                            (eox_api_response["EOXRecord"][0]["EOXMigrationDetails"]["MigrationProductId"] !=""):
                                eol_prod = eox_api_response["EOXRecord"][0]["EOXMigrationDetails"]["MigrationProductId"]
                                ldos_date = eox_api_response["EOXRecord"][0]["LastDateOfSupport"]["value"]
                            else:
                                replacement_is_ldos = False
                            
                        device["LDOSDate"] = ldos_date
                        
                        # Special case where Cat 9500 product SKU needs to be corrected
                        if eol_prod.startswith('C9500') and not(eol_prod.endswith('-A') or eol_prod.endswith('-E')):
                            eol_prod = eol_prod + '-A'                       
                        device["Replacement"] = eol_prod
                        LDOS_buffer[device["Product"]]= {"LDOSDate": device["LDOSDate"],"Replacement":device["Replacement"]}
                        
                # if there is no LDOS date for the product, then just enter blank for LDOS date and replacement        
                else :
                    device["LDOSDate"] = ""
                    device["Replacement"] = ""
                
                # Check if there is a ship date for the device.  This is used to calculate how long the device has been installed.
                if "shipDate" in dev:    
                    device["ShipDate"] = dev["shipDate"]
                else:
                    device["ShipDate"] = ""
                    
                # save the device LDOS info to the output file    
                # outfile.writelines(json.dumps(device)+"\n")


                output.append(device)
                
    return token,now,LDOS_buffer,output
                
# Function to determine how many Cisco API calls are needed.
def lookup_ldos():
    
    # opening the source file that has hte serial numbers and site ids.
    # opening the destinaton file to save the LDOS info
    infile = open(input_file,'r')
    
    output = []

    # Group serial numbers in blocks of 20.  This so we can call CCW-R API 20 serial numbers at a time.
    num_of_lines = len(open(input_file).readlines())
    a,b = divmod(num_of_lines, 20)
    
    # this is used to determine percentage completed for LDOS processing
    i = 0  
    
    # Get a new API token    
    token,now = getnewtoken()
    
    # This is used to remember if we have seen a product ID before.  This will reduce number of calls to EOX API
    LDOS_buffer={}

    # Get LDOS info in blocks of 20 serial numbers  
    for p in range(a):
        token,now,LDOS_buffer,output = get_ldos(20,infile,output,token,now,LDOS_buffer)
        i += 20
        # print out completion progress
        print(divmod(i*100,num_of_lines)[0],"%"," Complete")

    # Get LDOS info for the remainder after block of 20 have been processed
    if b != 0:
        token,now,LDOS_buffer,output = get_ldos(b,infile,output,token,now,LDOS_buffer)
    print("100%"," Complete")
                           
    return output
 





def plot_ldos_by_quantity(inventory,first_year,last_year):

    LDOS_sum = {}
    for year in range(first_year,last_year+1):
        num = 0
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                num += 1
        LDOS_sum[str(year)] = num
    
    # x-coordinates of left sides of bars  
    left = [i for i in range(len(LDOS_sum))]

    # heights of bars 
    height = [LDOS_sum[i] for i in LDOS_sum] 

    # labels for bars 
    tick_label = [i for i in LDOS_sum]

    plt.figure(figsize=(10,5))
    
    # plotting a bar chart 
    plt.bar(left, height, tick_label = tick_label, 
            width = 0.8, color = ['red', 'green']) 

    # naming the x-axis 
    plt.xlabel('Year',fontsize=26) 
    # naming the y-axis 
    plt.ylabel('Quantity',fontsize=26) 
    # plot title 
    plt.title('Number of LDOS Gear', fontsize=30)
    plt.savefig('LDOS-by-year.png')
    return

def plot_ldos_by_age(inventory,first_year,last_year):

    LDOS_sum = {}
    for year in range(first_year, last_year+1):
        years_shipped = []
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                if dev["ShipDate"] != "":
                    yr = int(dev["ShipDate"][0:4])
                    mo= int(dev["ShipDate"][5:7])
                    day= int(dev["ShipDate"][8:10])
                    f_date = date(yr, mo, day)
                    l_date = date(2020, 10, 19)
                    delta = l_date - f_date
                    years_shipped.append(delta.days/365)
        if years_shipped:
            LDOS_sum[str(year)] = round(statistics.median(years_shipped),2)
        else:
            LDOS_sum[str(year)] = 0
    
  
    
    # x-coordinates of left sides of bars  
    left = [i for i in range(len(LDOS_sum))]

    # heights of bars 
    height = [LDOS_sum[i] for i in LDOS_sum] 

    # labels for bars 
    tick_label = [i for i in LDOS_sum]

    plt.figure(figsize=(10,5))
    plt.axhline(y=6)
    
    # plotting a bar chart 
    plt.bar(left, height, tick_label = tick_label, 
            width = 0.8, color = ['red', 'green']) 

    # naming the x-axis 
    plt.xlabel('Year',fontsize=26) 
    # naming the y-axis 
    plt.ylabel('Age (in years)',fontsize=26) 
    # plot title 
    plt.title('Median Age of LDOS Gear', fontsize=30) 
    # function to show the plot 
    plt.savefig('LDOS-by-age.png') 
    return



def plot_ldos_by_site(inventory,first_year,last_year):

    LDOS_sum = {}
    for year in range(first_year, last_year+1):
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                if dev['Site'] in LDOS_sum:
                    if str(year) in LDOS_sum[dev['Site']]:
                        LDOS_sum[dev['Site']][str(year)] += 1
                    else:
                        LDOS_sum[dev['Site']][str(year)] = 1
                else:
                    LDOS_sum[dev['Site']] = {}
                    LDOS_sum[dev['Site']][str(year)] = 1
   

            
    labels = [i for i in LDOS_sum]
    y={}
    for i in LDOS_sum:
        for year in range(first_year, last_year+1):
            if str(year) in LDOS_sum[i]:
                if str(year) in y:
                    y[str(year)].append(LDOS_sum[i][str(year)])
                else:
                    y[str(year)] = []
                    y[str(year)].append(LDOS_sum[i][str(year)])
            elif str(year) in y:
                y[str(year)].append([0])
            else:
                y[str(year)] = [0]
   

    width = 0.35       # the width of the bars: can also be len(x) sequence


    fig, ax = plt.subplots(figsize=(10, 5))
    for year in y:
        ax.barh(labels, y[year], width,  label=year)


    
    ax.set_xlabel('Quantity',fontsize=26)
    ax.set_title('LDOS Quantity by Site', fontsize=30)
    ax.legend()
    plt.savefig('LDOS-by-site.png')

    return


def create_bom(replacement,first_year,last_year):
    
    from api_cred import c_token

    url = "https://api-test.cisco.com/commerce/EST/v2/POE/sync/createEstimate"
    
    line_num = 1
    p = {"ProcessQuote":{"DataArea":{"Quote":[{"QuoteHeader":{"Extension":[{"ID":[{"value":"","typeCode":"PriceListShortName"},{"value":"Resale","typeCode":"IntendedUseCode"}],"ValueText":[{"value":"LDOS-Test","typeCode":"Estimate Name"},{"value":"N","typeCode":"SweepsIndicator"}]}],"Status":[{"Code":{"value":"VALID","typeCode":"EstimateStatus"}}],"Party":[{"role":"End Customer","Name":[{"value":"ALEXANDER, O"}],"Location":[{"Address":[{"AddressLine":[{"value":"850 BRYANT ST","sequenceNumber":1},{"value":"","sequenceNumber":2},{"value":"","sequenceNumber":3}],"CityName":{"value":"SAN FRANCISCO"},"CountrySubDivisionCode":[{"value":"CA"}],"CountryCode":{"value":"US"},"PostalCode":{"value":"94103-4603"}}]}],"Contact":[{"PersonName":[{"PreferredName":[{"value":"TEST"}]}],"TelephoneCommunication":[{"Extension":[{"ValueText":[{"value":"12345","typeCode":"Contact Fax"}]}],"FormattedNumber":{"value":"987654321","typeCode":"Contact Phone"}}],"EMailAddressCommunication":[{"EMailAddressID":{"value":"asd123@gmail.com"}}]}]},{"role":"Install Site","Name":[{"value":"ONEIDA TRIBE OF WISCONSIN"}],"Location":[{"Address":[{"AddressLine":[{"value":"2170 AIRPORT DRIVE","sequenceNumber":1},{"value":"GAMING ADMINISTRATION","sequenceNumber":2},{"value":"","sequenceNumber":3}],"CityName":{"value":"GREEN BAY"},"CountrySubDivisionCode":[{"value":"WI"}],"CountryCode":{"value":"US"},"PostalCode":{"value":"54313"}}]}],"Contact":[{"PersonName":[{"PreferredName":[{"value":"Indus"}]}],"TelephoneCommunication":[{}],"EMailAddressCommunication":[{"EMailAddressID":{"value":"pqrs@cisco.com"}}]}]}],"BillToParty":{"ID":[{"value":"","typeCode":"BID"}],"Location":[{"Address":[{"AddressLine":[{"value":"8900 CANNOT SHIP ROAD","sequenceNumber":1}],"CityName":{"value":"LOS ANGELES"},"CountrySubDivisionCode":[{"value":"CA"}],"CountryCode":{"value":"US"},"PostalCode":{"value":"90017"}}]}]},"EffectiveTimePeriod":{"EndDateTime":"2013-08-04T07:00:00Z"},"QualificationTerm":[{}]},"QuoteLine":[\
        ]}]},"ApplicationArea":{"Sender":{"LogicalID":{"value":"847291911"},"ComponentID":{"value":"B2B-3.0"},"ReferenceID":{"value":"Tech Data"}},"Receiver":[{"LogicalID":{"value":"364132837"},"ID":[{"value":"Cisco Systems Inc."}]}],"CreationDateTime":"2017-01-07T21:00:59","BODID":{"value":"10-2056548"}}}}

    
    for year in range(first_year, last_year+1):
        for sku in replacement[str(year)]:
            if sku != "":
                line =  {"LineNumberID":{"value":""},"Status":[{"Reason":[{"value":"VALID"}]}],"Item":{"Extension":[{"Quantity":[{"value":0,"unitCode":"each"}]}],"ID":{"value":"","schemeAgencyID":"Cisco","typeCode":"PartNumber"},"Description":[{"value":"test"}],"Classification":[{"Extension":[{"ValueText":[{"value":"CONFIGURABLE","typeCode":"ItemType"}]}],"UNSPSCCode":{"value":"43222609"}}],"Specification":[{"Property":[{"Extension":[{"ValueText":[{"value":"1900-ISR-LOW-MODEL:PRODUCTNAME|CISCO1921-SEC/K9","typeCode":"ConfigurationPath"},{"value":"C1570456","typeCode":"ProductConfigurationReference"},{"value":"USER","typeCode":"ConfigurationSelectCode"},{"value":"N","typeCode":"BundleIndicator"},{"value":"INVALID","typeCode":"VerifiedConfigurationIndicator"},{"value":"2015-07-07T18:41:32Z","typeCode":"ValidationDateTime"}]}],"ParentID":{"value":"0"},"NameValue":{"value":"1.0","name":"CCWLineNumber"},"EffectiveTimePeriod":{"Duration":"P0Y0M35DT0H0M"}}]}]}}            
                line["LineNumberID"]["value"] = str(line_num)
                line_num += 1
                line["Item"]["Extension"][0]["Quantity"][0]["value"] = replacement[str(year)][sku]
                line["Item"]["ID"]["value"] = sku
                p["ProcessQuote"]["DataArea"]["Quote"][0]["QuoteLine"].append(line)

    if len(p["ProcessQuote"]["DataArea"]["Quote"][0]["QuoteLine"]) == 0:
        print("No BOM is built")
        return                  
    
    payload=json.dumps(p)
    print("c_token is ", c_token)
    ctoken = 'Bearer ' + c_token
    print(ctoken)

    headers = {
      'Accept': 'application/json',
      'Authorization': ctoken ,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data = payload)
    print(response.text.encode('utf8'))

    eresponse = json.loads(response.text)
    estimate = eresponse["AcknowledgeQuote"]['DataArea']['Quote'][0]['QuoteHeader']['ID']['value']

    return estimate


def main():
    # Delete preexisting webhooks created by this script
    delete_webhooks_with_name()

    create_webhooks(webhook_url)


    try:
        # Start the Flask web server
        flask_app.run(host="0.0.0.0", port=port)

    finally:
        print("Cleaning up webhooks...")
        delete_webhooks_with_name()


if __name__ == "__main__":
    main()


Creating Message Created Webhook...
Webex Teams Webhook:
{
  "id": "Y2lzY29zcGFyazovL3VzL1dFQkhPT0svZTBlYjQ5N2MtY2I4My00MzZjLWFiNjAtZmU0MDY3ZTc3Nzk5",
  "name": "botWithCardExampleWebhook",
  "targetUrl": "https://b3d07231ad41.ngrok.io/events",
  "resource": "messages",
  "event": "created",
  "orgId": "Y2lzY29zcGFyazovL3VzL09SR0FOSVpBVElPTi8xZWI2NWZkZi05NjQzLTQxN2YtOTk3NC1hZDcyY2FlMGUxMGY",
  "createdBy": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS81ZTc3ODRlYi0zMDE3LTQwMDMtYWRlZS0xYzgzMWY2MGFhZjM",
  "appId": "Y2lzY29zcGFyazovL3VzL0FQUExJQ0FUSU9OL0MzMmM4MDc3NDBjNmU3ZGYxMWRhZjE2ZjIyOGRmNjI4YmJjYTQ5YmE1MmZlY2JiMmM3ZDUxNWNiNGEwY2M5MWFh",
  "ownedBy": "creator",
  "status": "active",
  "created": "2020-10-29T13:16:05.837Z"
}
Webhook successfully created.
Creating Attachment Actions Webhook...
Webex Teams Webhook:
{
  "id": "Y2lzY29zcGFyazovL3VzL1dFQkhPT0svYzFlMzdjOTktMWRhNS00ZGQ4LTk5M2YtNGNiMmNkNDI4ZGJh",
  "name": "botWithCardExampleWebhook",
  "targetUrl": "https://b3d07231ad41.ngrok.io/events",
  

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)



            NEW MESSAGE IN ROOM 'Healkan Cheung'
            FROM 'Healkan Cheung'
            MESSAGE '['https://webexapis.com/v1/contents/Y2lzY29zcGFyazovL3VzL0NPTlRFTlQvZjRmZjZmMjAtMTllOC0xMWViLTkyY2UtZTc3NjhkYTdhMGM3LzA']'
            


127.0.0.1 - - [29/Oct/2020 09:16:32] "POST /events HTTP/1.1" 200 -



        NEW MESSAGE IN ROOM 'Healkan Cheung'
        FROM 'LDOS-Now'
        MESSAGE 'Preparing LDOS info...'
        
80 %  Complete
100%  Complete
[{'Site': 'TAMPA,US', 'Serial': 'SSI1714097S', 'Product': 'ASR1002-X', 'LDOSDate': '', 'Replacement': '', 'ShipDate': '2013-10-23T00:00:00Z'}, {'Site': 'ROANOKE,US', 'Serial': 'FOX1842G0ZM', 'Product': 'ASR1002-X=', 'LDOSDate': '', 'Replacement': '', 'ShipDate': '2020-10-06T00:00:00Z'}, {'Site': 'TEMPE,US', 'Serial': 'FDO1916E34D', 'Product': 'WS-C3650-48FWD-S', 'LDOSDate': '', 'Replacement': '', 'ShipDate': '2015-04-30T00:00:00Z'}, {'Site': 'TAMPA,US', 'Serial': 'FDO1916E3A3', 'Product': 'WS-C3650-48FWD-S', 'LDOSDate': '', 'Replacement': '', 'ShipDate': '2015-04-30T00:00:00Z'}, {'Site': 'SYRACUSE,US', 'Serial': 'FLM1937W0K0', 'Product': 'ISR4351-VSEC/K9', 'LDOSDate': '', 'Replacement': '', 'ShipDate': '2015-09-14T00:00:00Z'}, {'Site': 'TROY,US', 'Serial': 'JAE202603HU', 'Product': 'C1-C4500X-32SFP+', 'LDOSDate': '2025-10-31', 'Replacemen

...


            NEW MESSAGE IN ROOM 'Healkan Cheung'
            FROM 'LDOS-Now'
            MESSAGE '['https://webexapis.com/v1/contents/Y2lzY29zcGFyazovL3VzL0NPTlRFTlQvZDg3ZjlmMjAtMTlmMC0xMWViLTk4M2UtODU2Mjg4YTE5NGNkLzA']'
            
c_token is  0NLbJEQ6Qxpt3lICkyDHVawZuhWW
Bearer 0NLbJEQ6Qxpt3lICkyDHVawZuhWW


127.0.0.1 - - [29/Oct/2020 10:13:02] "POST /events HTTP/1.1" 200 -



            NEW MESSAGE IN ROOM 'Healkan Cheung'
            FROM 'LDOS-Now'
            MESSAGE '['https://webexapis.com/v1/contents/Y2lzY29zcGFyazovL3VzL0NPTlRFTlQvZGFhN2ZkNjAtMTlmMC0xMWViLWI5YjQtOWRiYjhmMTRlYTFmLzA']'
            
b'{"AcknowledgeQuote":{"DataArea":{"Acknowledge":{"ResponseCriteria":[{"ChangeStatus":{"Reason":[{"value":"Success"}]}}]},"Quote":[{"QuoteHeader":{"Extension":[{"ValueText":[{"value":"DWZHULDB","typeCode":"AccessKey"},{"value":"LDOS-Test","typeCode":"Estimate Name"}]}],"ID":{"value":"CC121472129GZ","typeCode":"Estimate ID"},"Status":[{"Reason":[{"value":"VALID"}]}]}}]},"ApplicationArea":{"Sender":{"LogicalID":{"value":""},"ReferenceID":{"value":"Tech Data"}},"Receiver":[{"ID":[{"value":"Tech Data"}]}],"CreationDateTime":"2020-10-29T14:13:04Z","BODID":{"value":"5389df3e-2ca5-4ff0-96f6-43cfbacaf6bb","schemeVersionID":"1.0","schemeAgencyID":"Cisco"}}}}'


127.0.0.1 - - [29/Oct/2020 10:13:05] "POST /events HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2020 10:13:06] "POST /events HTTP/1.1" 200 -



        NEW MESSAGE IN ROOM 'Healkan Cheung'
        FROM 'LDOS-Now'
        MESSAGE 'New CCW Estimate CC121472129GZ at https://tools.cisco.com/POE/Commerce/estimate#!&report=all '
        
